In [1]:
from hypex.dataset import Dataset, ExperimentData, InfoRole, TreatmentRole, TargetRole
from hypex.experiments.aa import AA_TEST
from hypex.reporters import AADictReporter

In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(int),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole()
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [3]:
data.data.dtypes

user_id            int32
signup_month       int32
treat              int32
pre_spends       float64
post_spends      float64
age              float64
gender          category
industry        category
dtype: object

In [25]:
test = AA_TEST
ed = ExperimentData(data)
result = test.execute(ed)

C:\Users\User\PycharmProjects\HypEx\hypex\dataset\backends\pandas_backend.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column_name] = self.data[column_name].astype(type_name)
C:\Users\User\PycharmProjects\HypEx\hypex\dataset\backends\pandas_backend.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column_name] = self.data[column_name].astype(type_name)
C:\Users\User\PycharmProjects\HypEx\hypex\dataset\backends\pandas_backend.py:83: SettingWithCopyWarning: 
A value is trying to

In [26]:
AADictReporter().report(result)

{'random_state': None,
 'B pre_spends control mean': 487.2096,
 'B pre_spends difference': -0.2317000000000462,
 'B pre_spends difference %': -0.04755653418980765,
 'B pre_spends test mean': 486.9779,
 'B post_spends control mean': 452.2280666666666,
 'B post_spends difference': -0.12702222222213777,
 'B post_spends difference %': -0.028088089082667356,
 'B post_spends test mean': 452.1010444444445,
 'B control size': 5000.0,
 'B control size %': 50.0,
 'B test size': 5000.0,
 'B test size %': 50.0,
 'TTest p-value': 0.7056285423808684,
 'TTest pass': 0.0,
 'KSTest p-value': 0.8651169116596868,
 'KSTest pass': 0.0,
 'mean test score': 0.8119541219000806}

In [28]:
result.analysis_tables

{'GroupSizes╰╰[][A]':                      B
 control size    5000.0
 control size %    50.0
 test size       5000.0
 test size %       50.0,
 "GroupDifference╰╰['pre_spends'][A]":                                   B
 pre_spends control mean  487.209600
 pre_spends difference     -0.231700
 pre_spends difference %   -0.047557
 pre_spends test mean     486.977900,
 "TTest╰╰['pre_spends'][A]":   group  statistic   p-value   pass
 0     B   0.613901  0.539295  False,
 "KSTest╰╰['pre_spends'][A]":   group  statistic   p-value   pass
 0     B     0.0128  0.807376  False,
 "GroupDifference╰╰['post_spends'][A]":                                    B
 post_spends control mean  452.228067
 post_spends difference     -0.127022
 post_spends difference %   -0.028088
 post_spends test mean     452.101044,
 "TTest╰╰['post_spends'][A]":   group  statistic   p-value   pass
 0     B   0.161171  0.871962  False,
 "KSTest╰╰['post_spends'][A]":   group  statistic   p-value   pass
 0     B      0.011  0.922

In [ ]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from hypex.comparators import TTest, KSTest, GroupSizes, GroupDifference
from hypex.utils import SpaceEnum
from hypex.splitters import AASplitter
from hypex.analyzers import OneAASplitAnalyzer
from hypex.experiments import CycledExperiment, Experiment, OnRoleExperiment

aa = Experiment(
    executors=[
        AASplitter(),
        OnRoleExperiment(
            executors=[
                GroupSizes(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                GroupDifference(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                TTest(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                KSTest(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
            ],
            role=TargetRole(),
        )
    ]
)
experiment = CycledExperiment(n_iterations=20, inner_executor=aa, analyzer=OneAASplitAnalyzer())
ed = ExperimentData(data)
res = experiment.execute(ed)

In [ ]:
res.additional_fields

In [ ]:
res.analysis_tables

In [ ]:
res_analyzer = OneAASplitAnalyzer().execute(res)

In [ ]:
res_analyzer.analysis_tables['OneAASplitAnalyzer╰╰']